In [1]:
class GBayes:
    
    def __init__(self,features,classes):
        
        numtrainData = len(features)
        numClasses = len(np.unique(classes))

        nums = np.zeros(numClasses)

        u = np.zeros((numClasses,features.shape[1]),float)
        var = np.zeros((numClasses,features.shape[1]),float)

        for i in range(numtrainData):
            nums[classes[i]] +=1
            u[classes[i]] += features[i]

        u = u/nums[:,np.newaxis]   

        for i in range(numtrainData):
            var[classes[i]] += (features[i] - u[classes[i]])**2

        var = var / nums[:,np.newaxis]
        
        self.u = u
        self.var = var
        self.numClasses = len(np.unique(classes))
        
    def predict(self,f):
        
        p = np.zeros((self.numClasses),float)
        r = []
        
        for x in f :
            for i in range(len(p)):
                p[i] = (1.0/np.sqrt(2*np.pi*(np.linalg.norm(self.var[i]))))*np.exp(-(np.linalg.norm((x-self.u[i])**2))/(2*np.linalg.norm(self.var[i])))

            r.append(np.argmax(p))

        return np.array(r)

In [2]:
import numpy as np
import csv


data = []
for k in range(10):
   
    with open('Data/otto%i.csv'%k, 'rb') as trainFile:
        reader = csv.reader(trainFile)
        i = 0
        reader.next()
        for r in reader:
            data.append((np.array((r[:len(r) - 1])).astype(float),np.array(r[len(r) - 1]).astype(float).astype(int)))
            
        
datanum = len(data)
trainData = data[:datanum - datanum/10]
trainClasses = np.array([c for _,c in trainData]) 
trainFeatures = np.array([x for x,_ in trainData])
testData = data[datanum - datanum/10:datanum]
testFeatures = np.array([x for x,_ in testData])
testClasses = np.array([c for _,c in testData]) 

testclassesNum = len(set(trainClasses))
testfeaturesNum = testFeatures.shape[0]

In [3]:
model = GBayes(trainFeatures,trainClasses)
pred = model.predict(testFeatures)


In [4]:
from sklearn import metrics 
print("Classification report for classifie" , metrics.accuracy_score(testClasses,pred))

('Classification report for classifie', 0.40747330960854095)


# Cross Validation

In [7]:
from sklearn import metrics
precisions = np.zeros(9)
divInd =  datanum/10
data = np.concatenate((data[:datanum - datanum/10],data[datanum - datanum/10:datanum]),axis=0)

for j in range(9):
    trainData = np.concatenate((data[:divInd*j],data[divInd*j+divInd:]))
    testData = data[divInd*j:divInd*j+divInd]
    
    
    trainFeatures = np.array([x for x,_ in trainData])
    trainClasses = np.array([c for _,c in trainData]) 

    trainFeatures = trainFeatures
    classesNum = len(set(trainClasses))
    featuresNum = trainFeatures.shape[0]

    trainTargets = np.zeros((featuresNum,classesNum))

    for i in range(featuresNum):
        trainTargets[i,trainClasses[i]] = 1

    model = GBayes(trainFeatures,trainClasses)
    
    testFeatures = np.array([x for x,_ in testData])
    testClasses = np.array([c for _,c in testData]) 

    pred = model.predict(testFeatures)
   
    precisions[j] = metrics.accuracy_score(testClasses,pred)
    
print np.mean(precisions)

0.40991768216
